In [1]:
import numpy as np 
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [3]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(
    num_words=None
)
X_train.shape, X_test.shape

17465344/17464789 [==============================] - 0s 0us/step


((25000,), (25000,))

In [4]:
index_dict = {}
for key, value in imdb.get_word_index().items():
    index_dict[value] = key
len(index_dict)

1646592/1641221 [==============================] - 0s 0us/step


88584

In [5]:
# 뉴스 기사의 길이
print('영화평 최대 길이:', max(len(l) for l in X_train))
print('영화평 평균 길이:', sum(map(len, X_train))/len(X_train))

영화평 최대 길이: 2494
영화평 평균 길이: 238.71364


In [6]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
# 데이터 불러옴
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)  # 단어 빈도수: 5000

In [8]:
# 패딩
max_len = 500 # 문장의 단어수 : 500단어
X_train = pad_sequences(X_train,maxlen= max_len)
X_test = pad_sequences(X_test, maxlen= max_len)

In [9]:
from sklearn.model_selection import train_test_split

X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, stratify= y_test, test_size = 0.4, random_state=seed
)
X_test.shape, X_val.shape, y_test.shape, y_val.shape

((15000, 500), (10000, 500), (15000,), (10000,))

## 모델 정의 / 설정 / 학습 / 평가

In [11]:
model = Sequential([
    Embedding(5000,120),
    LSTM(120),
    Dense(1, activation = 'sigmoid')

])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 120)         600000    
_________________________________________________________________
lstm (LSTM)                  (None, 120)               115680    
_________________________________________________________________
dense (Dense)                (None, 1)                 121       
Total params: 715,801
Trainable params: 715,801
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(
    optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy']
)

In [14]:
# Callback 함수
import os
if not os.path.exists('model'):
  os.mkdir('model')
model_file = 'model/best_imdb_conv1d_lstm.h5'
mc = ModelCheckpoint(model_file, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [15]:
history = model.fit(
    X_train, y_train, batch_size=100, epochs=50,
    validation_data=(X_val, y_val), verbose=0, callbacks=[mc, es]
)


Epoch 00001: val_loss improved from inf to 0.31645, saving model to model/best_imdb_conv1d_lstm.h5

Epoch 00002: val_loss did not improve from 0.31645

Epoch 00003: val_loss improved from 0.31645 to 0.31215, saving model to model/best_imdb_conv1d_lstm.h5

Epoch 00004: val_loss did not improve from 0.31215

Epoch 00005: val_loss did not improve from 0.31215

Epoch 00006: val_loss did not improve from 0.31215

Epoch 00007: val_loss did not improve from 0.31215

Epoch 00008: val_loss did not improve from 0.31215


In [16]:
best_model = load_model(model_file)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 5s 10ms/step - loss: 0.3144 - accuracy: 0.8727


[0.3144129812717438, 0.8727333545684814]